---
title: "PS5"
author: "Yuliana and Dale"
date: "date"
format: 
  pdf:
    include-in-header: 
       text: |
         \usepackage{fvextra}
         \DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines,commandchars=\\\{\}}
include-before-body:
  text: |
    \RecustomVerbatimEnvironment{verbatim}{Verbatim}{
      showspaces = false,
      showtabs = false,
      breaksymbolleft={},
      breaklines
    }
output:
  echo: false
  eval: false
---


**Due 11/9 at 5:00PM Central. Worth 100 points + 10 points extra credit.**

## Submission Steps (10 pts)
1. This problem set is a paired problem set.
2. Play paper, scissors, rock to determine who goes first. Call that person *Partner 1*.
    - Partner 1 (name and cnet ID): Yuliana Zhang ; yuejiu
    - Partner 2 (name and cnet ID): Dale (Yuanhao) Jin; jin86
3. Partner 1 will accept the `ps5` and then share the link it creates with their partner. You can only share it with one partner so you will not be able to change it after your partner has accepted. 
4. "This submission is our work alone and complies with the 30538 integrity policy." Add your initials to indicate your agreement: \*\* YZ\*\* \*\* Dale Jin\*\*
5. "I have uploaded the names of anyone else other than my partner and I worked with on the problem set **[Yuqing Wen](https://docs.google.com/forms/d/185usrCREQaUbvAXpWhChkjghdGgmAZXA3lPWpXLLsts/edit)**"  (1 point)
6. Late coins used this pset: \*\* 0 \*\* Late coins left after submission: \*\* 2 \*\*
7. Knit your `ps5.qmd` to an PDF file to make `ps5.pdf`, 
    * The PDF should not be more than 25 pages. Use `head()` and re-size figures when appropriate. 
8. (Partner 1): push  `ps5.qmd` and `ps5.pdf` to your github repo.
9. (Partner 1): submit `ps5.pdf` via Gradescope. Add your partner on Gradescope.
10. (Partner 1): tag your submission in Gradescope

\newpage


In [ ]:
import pandas as pd
import altair as alt
import time

import warnings 
warnings.filterwarnings('ignore')
alt.renderers.enable("png")

import requests
from bs4 import BeautifulSoup

from datetime import datetime
from urllib.parse import urljoin
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt

## Step 1: Develop initial scraper and crawler

### 1. Scraping (PARTNER 1)


In [ ]:
# First, get the url
url = 'https://oig.hhs.gov/fraud/enforcement/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

# Initialize lists to store extracted information
titles = []
dates = []
categories = []
links = []

# Find each enforcement action entry
for item in soup.find_all('li', class_ = 'usa-card card--list pep-card--minimal mobile:grid-col-12'):
    # Extract title and link
    title_tag = item.find('h2', class_ = 'usa-card__heading').find('a')
    title = title_tag.text.strip()
    link = title_tag['href']
    
    titles.append(title)
    links.append(f'https://oig.hhs.gov{link}')  # form the full url

    # Extract date
    date = item.find('span', class_ = 'text-base-dark padding-right-105').text.strip()
    dates.append(date)

    # Extract category
    category_tag = item.find('ul', class_ = 'display-inline add-list-reset').find('li')
    category = 'N/A'
    category = category_tag.text.strip()
    categories.append(category)
    data = pd.DataFrame({
    'Title': titles,
    'Date': dates,
    'Category': categories,
    'Link': links
})

# Display the dataframe
print(data.head())

### 2. Crawling (PARTNER 1)


In [ ]:
# Part 2: Adding the Agency Name by Crawling Each Link
agencies = []

# Get loop through each detailed page
for link in data['Link']:
    response = requests.get(link)
    detail_soup = BeautifulSoup(response.content, 'lxml')
    agency_name = 'N/A'
    
    # Locate the <ul> tag containing the details
    details_list = detail_soup.find('ul', class_ = 'usa-list usa-list--unstyled margin-y-2')
    if details_list:
        for li in details_list.find_all('li'):
            label_span = li.find('span', class_ = 'padding-right-2 text-base')
            if label_span:
                label_text = label_span.text.strip()
                # Check if the label is "Date:" or "Agency:"
                if label_text == "Agency:":
                    agency_name = label_span.find_next_sibling(text=True).strip()
    # Append extracted data to lists
    agencies.append(agency_name)

# Display the updated DataFrame
data['Agency'] = agencies
print(data.head())

## Step 2: Making the scraper dynamic

### 1. Turning the scraper into a function 

* a. Pseudo-Code (PARTNER 2)
1. start function and taking month+year as inputs
2. if year is < 2013, return false
3. if year is >= 2013
  a. create empty lists to store title, date, category, agency, link
  b. create a int for page number
  c. while loop on page
      - for loop on item
        - check if extract date match the input, 
        - if yes, break
        - extract info
        - go to detail page and extract agency 
  d. wait 1 second and flip

* b. Create Dynamic Scraper (PARTNER 2)


In [ ]:
def ScrapeEnforcement(month, year):
  input_date = datetime(year, month, 1)

  if year < 2013:
    print("Please enter a year of 2013 or later. Data is only available from 2013 onward.")
    return None

  base_url = 'https://oig.hhs.gov/fraud/enforcement/'

  # create lists to store info
  titles = []
  dates = []
  categories = []
  links = []
  agencies = []

  page = 1

  # loop over pages
  while True:
    url = f"{base_url}?page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # loop over enforcements
    for item in soup.find_all('li', class_ = 'usa-card card--list pep-card--minimal mobile:grid-col-12'):
      # checking and compareing date      
      date_str = item.find('span', class_ = 'text-base-dark padding-right-105').text.strip()
      date = datetime.strptime(date_str, "%B %d, %Y") # convert to datetime

      if date < input_date:
        df_enforcement = pd.DataFrame({
          'Title': titles,
          'Date': dates,
          'Category': categories,
          'Link': links,
          'Agency': agencies
        })
        csv = f"enforcement_actions_year_month.csv"
        df_enforcement.to_csv(csv, index = False)
        return df_enforcement

      # Extract date
      dates.append(date)

      # Extract title
      title_tag = item.find('h2', class_ = 'usa-card__heading').find('a')
      title = title_tag.text.strip()
      titles.append(title)

      # Extract link
      link = title_tag['href']
      full_link = urljoin('https://oig.hhs.gov', link) # Form the full URL
      links.append(full_link)  

      # Extract category
      category_tag = item.find('ul', class_ = 'display-inline add-list-reset').find('li')
      category = 'N/A'
      category = category_tag.text.strip()
      categories.append(category)

      # go to detail page
      detail_response = requests.get(full_link)
      detail_soup = BeautifulSoup(detail_response.content, 'lxml')
      agency_name = 'N/A'
        
      # Locate the <ul> tag containing the details
      details_list = detail_soup.find('ul', class_ = 'usa-list usa-list--unstyled margin-y-2')
      if details_list:
        for li in details_list.find_all('li'):
          label_span = li.find('span', class_ = 'padding-right-2 text-base')
          if label_span:
            label_text = label_span.text.strip()
            # Check if the label is "Date:" or "Agency:"
            if label_text == "Agency:":
              agency_name = label_span.find_next_sibling(text = True).strip()
              break
      # Append extracted data to lists
      agencies.append(agency_name)
      
    # stop and flip the page
    page += 1
    time.sleep(1)

  df_enforcement = pd.DataFrame({
    'Title': titles,
    'Date': dates,
    'Category': categories,
    'Link': links,
    'Agency': agencies
  })
  csv = f"enforcement_actions_year_month.csv"
  df_enforcement.to_csv(csv, index = False)
  return df_enforcement

In [ ]:
# collecting the enforcement actions since January 2023
df_2023 = ScrapeEnforcement(1, 2023)

In [ ]:
print(len(df), 'enforcement actions since January 2023')

earlist_enforcement = df_2023.iloc[-1]
print('The date and details of the earliest enforcement action it scraped is', earlist_enforcement)

* c. Test Partner's Code (PARTNER 1)


In [ ]:
# collecting the enforcement actions since January 2021
df_2021 = ScrapeEnforcement(1, 2021)

In [ ]:
print(len(df_2021), 'enforcement actions since January 2021')

earlist_enforcement = df_2021.iloc[-1]
print('The date and details of the earliest enforcement action it scraped is', earlist_enforcement)

## Step 3: Plot data based on scraped data

### 1. Plot the number of enforcement actions over time (PARTNER 2)


In [ ]:
# Extract year month
df_2021['Date'] = pd.to_datetime(df_2021['Date'])
df_2021['yearmonth'] = df_2021['Date'].dt.strftime('%Y-%m')

enforcement_counts = df_2021.groupby('yearmonth').size().reset_index(name = 'Count')

# Convert YearMonth to a string format
enforcement_counts['yearmonth'] = enforcement_counts['yearmonth'].astype(str)

alt.Chart(enforcement_counts).mark_line().encode(
    x = 'yearmonth(yearmonth):T',
    y = 'Count:Q',
    tooltip = ['yearmonth', 'Count']
).properties(
    title = 'Number of Enforcement Actions Over Time (Monthly)',
    width = 400,
    height = 200
)

### 2. Plot the number of enforcement actions categorized: (PARTNER 1)

* based on "Criminal and Civil Actions" vs. "State Enforcement Agencies"


In [ ]:
# Filter the data to include only the two specified categories
filtered_df = df_2021[df_2021['Category'].isin(["Criminal and Civil Actions", "State Enforcement Agencies"])]

# Group by'Category' and count the number of actions 
actions_counts = filtered_df.groupby(['yearmonth','Category']).size().reset_index(name = 'Count')

# Plotting with Altair
alt.Chart(actions_counts).mark_line().encode(
    x = alt.X('yearmonth(yearmonth):T', title = 'Time'),
    y = alt.Y('Count:Q', title = 'Number of Enforcement Actions'),
    color = 'Category:N',
).properties(
    title = '"Criminal and Civil Actions" vs. "State Enforcement Agencies" Over Time',
    width = 600,
    height = 300
)

* based on five topics


In [ ]:
# Filter to include only "Criminal and Civil Actions" 
five_topics_df = df_2021[df_2021['Category'] == "Criminal and Civil Actions"]

# Define a function to assign topics 
def assign_topic(title):
    title = title.lower()  
    if "bribery" in title or "corruption" in title or "kickback" in title:
        return "Bribery/Corruption"
    elif "financial" in title or "bank" in title or "business" in title or "billing" in title or "money" in title or "investment" in title:
        return "Financial Fraud"
    elif "drug" in title or "opioid" in title or "narcotics" in title:
        return "Drug Enforcement"
    elif "medica" in title or "health" in title:
        return "Health Care Fraud"
    else:
        return "Other"

# Apply the topic assignment function to each title
five_topics_df['Topic'] = five_topics_df['Title'].apply(assign_topic)

# Group by 'yearmonth' and 'Topic' and count the number of actions per topic per month
topic_counts = five_topics_df.groupby(['yearmonth', 'Topic']).size().reset_index(name = 'Count')

# Plotting with Altair
alt.Chart(topic_counts).mark_line().encode(
    x = alt.X('yearmonth(yearmonth):T', title = 'Date'),
    y = alt.Y('Count:Q', title = 'Number of Enforcement Actions'),
    color = 'Topic:N'
).properties(
    title = 'Enforcement Actions by Topic within "Criminal and Civil Actions"',
    width = 600,
    height = 300
)

## Step 4: Create maps of enforcement activity

### 1. Map by State (PARTNER 1)


In [ ]:
# filter states
states_count = df_2021[df_2021['Agency'].str.contains("State of", case=False, na=False)]
len(df_2021)
# clean the state names
# Attribution: I ask ChatGPT how to extract district name
states_count['NAME'] = states_count['Agency'].str.extract(r"State of (.+)", expand = False)

# group by state
states_count = states_count.groupby('NAME').size().reset_index(name = 'Count')

In [ ]:
states_map = gpd.read_file('cb_2018_us_state_500k.shp')

# merge them with the shapefile
states = states_map.merge(states_count, left_on = 'NAME', right_on = 'NAME', how = 'left')

# plot a choropleth of the number of enforcement actions for each state
states.plot(column = "Count", legend = True).set_axis_off()

### 2. Map by District (PARTNER 2)


In [ ]:
# filter districts
districts_count = df_2021[df_2021['Agency'].str.contains("District", case = False, na = False)]

# clean the district names
# Attribution: I ask ChatGPT how to extract district name
districts_count['Judicial District'] = districts_count['Agency'].str.extract(r",\s*(.*District of .*)")

# group by districts
districts_count = districts_count.groupby('Judicial District').size().reset_index(name = 'Count')
districts_count

In [ ]:
districts_map = gpd.read_file('US_Attorney_Districts_Shapefile_simplified_20241107.csv')

# Convert csv to geometry
districts_map['the_geom'] = districts_map['the_geom'].apply(wkt.loads)
districts_map = gpd.GeoDataFrame(districts_map, geometry = 'the_geom')

# merge them with the shapefile
districts = districts_map.merge(districts_count, left_on = 'Judicial District', right_on = 'Judicial District', how = 'left')

# plot a choropleth of the number of enforcement actions in each US Attorney District
districts.plot(column = "Count", legend = True).set_axis_off()

## Extra Credit

### 1. Merge zip code shapefile with population

### 2. Conduct spatial join

### 3. Map the action ratio in each district